In [ ]:
import numpy as np
from numpy import shape

from datetime import date

import random
import math
import dill
import glob
import gc

# from tensorflow import keras

import matplotlib.pyplot as plt

import tensorflow as tf

from random import seed

# Import train_test_split function
from sklearn.model_selection import train_test_split

# Quelle: https://towardsdatascience.com/building-a-convolutional-neural-network-cnn-in-keras-329fbbadc5f5

# TODO Überarbeiten, welche Funktionen benötigt werden
import tensorflow as tf
from tensorflow import device

# Tensorflow soll auf CPU und nicht auf der GPU laufen
device("cpu:0")

# für GPU:
# tf.device("gpu:0")

In [2]:
%run '/home/jovyan/rna/_functions/functions.py'

# RNASeq Daten

In [3]:
path_file = "/home/jovyan/rna/rna/data/agg_gene_data/agg_gene_data_short_ALL.csv"

X_genes = pd.read_csv(
        path_file, 
        header = 0,
        index_col = 0
    )

X_genes = X_genes.drop("DIAGNOSIS", axis=1)

# Subjects metadata
X_metadata_path = "/home/jovyan/rna/rna/data/agg_gene_data/agg_gene_metadata_ALL.csv"

X_metadata = pd.read_csv(
        X_metadata_path, 
        header = 0
)

X_genes = X_genes.merge(X_metadata[["sample", "DIAGNOSIS"]], "left", left_on = "id", right_on = "sample")

# Extract dependend variable
Y = X_genes["DIAGNOSIS"]
Y.value_counts()

PD         367
Control    171
Name: DIAGNOSIS, dtype: int64

In [4]:
# Labels faktorisieren
Y = pd.factorize(Y)[0] # Aus bisheriger Analyse und als integer gecasted

print("0 = Control\n1 = PD")
print(np.unique(Y, return_counts=True))

X_genes = X_genes.drop(["sample","DIAGNOSIS", "id"], axis = 1)

0 = Control
1 = PD
(array([0, 1]), array([171, 367]))


### Load Persistence Landscapes

In [5]:
pl_folderpath = "/home/jovyan/rna/rna/data/persistence_landscapes_averages/"

In [6]:
# Eingelesenen Dateien
print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledWithin*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledWithin*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledBetween*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledBetween*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_unscaled*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_unscaled*.pkl")[-1])

avgPL_bucket01_H0_scaledWithin = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledWithin*.pkl")[-1])
avgPL_bucket01_H1_scaledWithin = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledWithin*.pkl")[-1])

avgPL_bucket01_H0_scaledBetween = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledBetween*.pkl")[-1])
avgPL_bucket01_H1_scaledBetween = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledBetween*.pkl")[-1])

avgPL_bucket01_H0_unscaled = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_unscaled*.pkl")[-1])
avgPL_bucket01_H1_unscaled = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_unscaled*.pkl")[-1])

/home/jovyan/rna/rna/data/persistence_landscapes_averages/avgPL_bucket01_H0_scaledWithin_2022-03-05.pkl
/home/jovyan/rna/rna/data/persistence_landscapes_averages/avgPL_bucket01_H1_scaledWithin_2022-03-03.pkl
/home/jovyan/rna/rna/data/persistence_landscapes_averages/avgPL_bucket01_H0_scaledBetween_2022-03-04.pkl
/home/jovyan/rna/rna/data/persistence_landscapes_averages/avgPL_bucket01_H1_scaledBetween_2022-03-03.pkl
/home/jovyan/rna/rna/data/persistence_landscapes_averages/avgPL_bucket01_H0_unscaled_2022-03-03.pkl
/home/jovyan/rna/rna/data/persistence_landscapes_averages/avgPL_bucket01_H1_unscaled_2022-03-03.pkl


### Train-Test-Split Perisistence Landscapes

In [7]:
# Bucket01 scaledBetween
seed(999)
# Train-Test-Split 
X_b01_scaledBetween_H0_train, X_b01_scaledBetween_H0_test, Y_b01_scaledBetween_H0_train, Y_b01_scaledBetween_H0_test = train_test_split(avgPL_bucket01_H0_scaledBetween,
                                                                                                                                        Y,
                                                                                                                                        test_size=0.2)

seed(999)
# Train-Test-Split 
X_b01_scaledBetween_H1_train, X_b01_scaledBetween_H1_test, Y_b01_scaledBetween_H1_train, Y_b01_scaledBetween_H1_test = train_test_split(avgPL_bucket01_H1_scaledBetween,
                                                                                                                                        Y,
                                                                                                                                        test_size=0.2)
# Bucket01 scaledWithin
seed(999)
# Train-Test-Split 
X_b01_scaledWithin_H0_train, X_b01_scaledWithin_H0_test, Y_b01_scaledWithin_H0_train, Y_b01_scaledWithin_H0_test = train_test_split(avgPL_bucket01_H0_scaledWithin,
                                                                                                                                    Y,
                                                                                                                                    test_size=0.2)
seed(999)
# Train-Test-Split 
X_b01_scaledWithin_H1_train, X_b01_scaledWithin_H1_test, Y_b01_scaledWithin_H1_train, Y_b01_scaledWithin_H1_test = train_test_split(avgPL_bucket01_H1_scaledWithin,
                                                                                                                                    Y,
                                                                                                                                    test_size=0.2)

# Bucket01 Unscaled
seed(999)
# Train-Test-Split 
X_b01_unscaled_H0_train, X_b01_unscaled_H0_test, Y_b01_unscaled_H0_train, Y_b01_unscaled_H0_test = train_test_split(avgPL_bucket01_H0_unscaled,
                                                                                                                    Y,
                                                                                                                    test_size=0.2)
seed(999)
# Train-Test-Split 
X_b01_unscaled_H1_train, X_b01_unscaled_H1_test, Y_b01_unscaled_H1_train, Y_b01_unscaled_H1_test = train_test_split(avgPL_bucket01_H1_unscaled,
                                                                                                                    Y,
                                                                                                                    test_size=0.2)


### Train-Test-Split raw data

In [12]:
# Setze Seed, damit der Train-Test-Split bei den Gene-Daten der Gleiche ist wie bei den PL
seed(999)

# Split dataset into training set and test set
X_raw_train, X_raw_test, y_raw_train, y_raw_test = train_test_split(X_genes, Y, test_size=0.2)

print("Anteil PD-Ausprägungen in Gesamtdaten: ", sum(Y)/len(Y))
print("Anteil PD-Ausprägungen in Trainingsdaten: ", sum(Y_train)/len(Y_train))
print("Anteil PD-Ausprägungen in Testdaten: ", sum(Y_test)/len(Y_test))

Anteil PD-Ausprägungen in Gesamtdaten:  0.6821561338289963
Anteil PD-Ausprägungen in Trainingsdaten:  0.6744186046511628
Anteil PD-Ausprägungen in Testdaten:  0.7129629629629629


# Machine Learning Classification

## Support Vector Machine

In [9]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [10]:
folderpath_results = "/home/jovyan/rna/rna/results/"

##### SVM (raw data)

In [29]:
# Linear - Rawdata
clf = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_raw_train, y_raw_train.ravel())

print("Accuracy SVM on raw data: " + str(clf.score(X_raw_test, y_raw_test))[:10])

evaluation(y_test = y_raw_test,
           y_pred = clf.predict(X_raw_test),
           filename_csv = folderpath_results + "svm_linear_rawdata.csv")

Accuracy SVM on raw data: 0.66666666
Confusion-Matrix:
 [[11 29]
 [ 7 61]] 
 
True Negative:  11 
True Positive 61 
False Negative 7 
False Positive 29

TPR: 0.8970588235294118 
TNR:  0.275

Accuracy: 0.6666666666666666
F1 (micro): 0.6666666666666666
F1 (macro): 0.5757311217808817


In [42]:
# RBF - Rawdata
clf = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_raw_train, y_raw_train.ravel())

print("Accuracy SVM on raw data: " + str(clf.score(X_raw_test, y_raw_test))[:10])

evaluation(y_test = y_raw_test,
           y_pred = clf.predict(X_raw_test),
           filename_csv = folderpath_results + "svm_rbf_rawdata.csv")

Accuracy SVM on raw data: 0.62962962
Confusion-Matrix:
 [[ 0 40]
 [ 0 68]] 
 
True Negative:  0 
True Positive 68 
False Negative 0 
False Positive 40

TPR: 1.0 
TNR:  0.0

Accuracy: 0.6296296296296297
F1 (micro): 0.6296296296296297
F1 (macro): 0.38636363636363635


##### SVM (Persistent Landscapes  - scaledWithin)

In [19]:
# Linear - H0 - scaledWithin
clf_pl_h0 = make_pipeline(StandardScaler(), 
                       SVC(kernel='linear')).fit(X_b01_scaledWithin_H0_train, Y_b01_scaledWithin_H0_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl_h0.score(X_b01_scaledWithin_H0_test, Y_b01_scaledWithin_H0_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H0_test,
           y_pred = clf_pl_h0.predict(X_b01_scaledWithin_H0_test),
           filename_csv = folderpath_results + "svm_linear_H0_scaledWithin.csv")

Accuracy SVM on peristence landscapes (scaled within sample): 0.62037037
Confusion-Matrix:
 [[ 5 21]
 [20 62]] 
 
True Negative:  5 
True Positive 62 
False Negative 20 
False Positive 21

TPR: 0.7560975609756098 
TNR:  0.19230769230769232

Accuracy: 0.6203703703703703
F1 (micro): 0.6203703703703703
F1 (macro): 0.47379679144385023


In [30]:
# Linear - H1 - scaledWithin
clf_pl = make_pipeline(StandardScaler(), 
                       SVC(kernel='linear')).fit(X_b01_scaledWithin_H1_train, Y_b01_scaledWithin_H1_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl.score(X_b01_scaledWithin_H1_test, Y_b01_scaledWithin_H1_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H1_test,
           y_pred = clf_pl.predict(X_b01_scaledWithin_H1_test),
           filename_csv = folderpath_results + "svm_linear_H1_scaledWithin.csv")

Accuracy SVM on peristence landscapes (scaled within sample): 0.53703703
Confusion-Matrix:
 [[14 24]
 [26 44]] 
 
True Negative:  14 
True Positive 44 
False Negative 26 
False Positive 24

TPR: 0.6285714285714286 
TNR:  0.3684210526315789

Accuracy: 0.5370370370370371
F1 (micro): 0.5370370370370371
F1 (macro): 0.4983277591973244


In [31]:
# RBF - H0 - scaledWithin
clf_pl_h0 = make_pipeline(StandardScaler(), 
                       SVC(kernel='rbf')).fit(X_b01_scaledWithin_H0_train, Y_b01_scaledWithin_H0_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl_h0.score(X_b01_scaledWithin_H0_test, Y_b01_scaledWithin_H0_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H0_test,
           y_pred = clf_pl_h0.predict(X_b01_scaledWithin_H0_test),
           filename_csv = folderpath_results + "svm_rbf_H0_scaledWithin.csv")

Accuracy SVM on peristence landscapes (scaled within sample): 0.75925925
Confusion-Matrix:
 [[ 0 26]
 [ 0 82]] 
 
True Negative:  0 
True Positive 82 
False Negative 0 
False Positive 26

TPR: 1.0 
TNR:  0.0

Accuracy: 0.7592592592592593
F1 (micro): 0.7592592592592593
F1 (macro): 0.43157894736842106


In [32]:
# RBF - H1 - scaledWithin
clf_pl_rbf = make_pipeline(StandardScaler(), 
                       SVC(kernel='rbf')).fit(X_b01_scaledWithin_H1_train, Y_b01_scaledWithin_H1_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl_rbf.score(X_b01_scaledWithin_H1_test, Y_b01_scaledWithin_H1_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H1_test,
           y_pred = clf_pl_rbf.predict(X_b01_scaledWithin_H1_test),
           filename_csv = folderpath_results + "svm_rbf_H1_scaledWithin.csv")

Accuracy SVM on peristence landscapes (scaled within sample): 0.64814814
Confusion-Matrix:
 [[ 0 38]
 [ 0 70]] 
 
True Negative:  0 
True Positive 70 
False Negative 0 
False Positive 38

TPR: 1.0 
TNR:  0.0

Accuracy: 0.6481481481481481
F1 (micro): 0.6481481481481481
F1 (macro): 0.39325842696629215


##### SVM (Persistent Landscapes - scaledBetween)

In [34]:
# Linear - H0 - scaledBetween
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_scaledBetween_H0_train, Y_b01_scaledBetween_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_scaledBetween_H0_test, Y_b01_scaledBetween_H0_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H0_test,
           y_pred = clf_pl.predict(X_b01_scaledBetween_H0_test),
           filename_csv = folderpath_results + "svm_linear_H0_scaledBetween.csv")

Accuracy SVM : 0.57407407
Confusion-Matrix:
 [[ 0 42]
 [ 4 62]] 
 
True Negative:  0 
True Positive 62 
False Negative 4 
False Positive 42

TPR: 0.9393939393939394 
TNR:  0.0

Accuracy: 0.5740740740740741
F1 (micro): 0.5740740740740741
F1 (macro): 0.36470588235294116


In [35]:
# Linear - H1 - scaledBetween
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_scaledBetween_H1_train, Y_b01_scaledBetween_H1_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_scaledBetween_H1_test, Y_b01_scaledBetween_H1_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H1_test,
           y_pred = clf_pl.predict(X_b01_scaledBetween_H1_test),
           filename_csv = folderpath_results + "svm_linear_H1_scaledBetween.csv")

Accuracy SVM : 0.65740740
Confusion-Matrix:
 [[11 25]
 [12 60]] 
 
True Negative:  11 
True Positive 60 
False Negative 12 
False Positive 25

TPR: 0.8333333333333334 
TNR:  0.3055555555555556

Accuracy: 0.6574074074074074
F1 (micro): 0.6574074074074074
F1 (macro): 0.5686062830616431


In [36]:
# RBF - H0 - scaledBetween
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_scaledBetween_H0_train, Y_b01_scaledBetween_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_scaledBetween_H0_test, Y_b01_scaledBetween_H0_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H0_test,
           y_pred = clf_pl.predict(X_b01_scaledBetween_H0_test),
           filename_csv = folderpath_results + "svm_rbf_H0_scaledBetween.csv")

Accuracy SVM : 0.61111111
Confusion-Matrix:
 [[ 0 42]
 [ 0 66]] 
 
True Negative:  0 
True Positive 66 
False Negative 0 
False Positive 42

TPR: 1.0 
TNR:  0.0

Accuracy: 0.6111111111111112
F1 (micro): 0.6111111111111112
F1 (macro): 0.37931034482758624


In [37]:
# RBF - H1 - scaledBetween
clf_pl_rbf = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_scaledBetween_H1_train, Y_b01_scaledBetween_H1_train)

print("Accuracy SVM : " + str(clf_pl_rbf.score(X_b01_scaledBetween_H1_test, Y_b01_scaledBetween_H1_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H1_test,
           y_pred = clf_pl_rbf.predict(X_b01_scaledBetween_H1_test),
           filename_csv = folderpath_results + "svm_rbf_H1_scaledBetween.csv")

Accuracy SVM : 0.66666666
Confusion-Matrix:
 [[ 0 36]
 [ 0 72]] 
 
True Negative:  0 
True Positive 72 
False Negative 0 
False Positive 36

TPR: 1.0 
TNR:  0.0

Accuracy: 0.6666666666666666
F1 (micro): 0.6666666666666666
F1 (macro): 0.4


##### SVM (Persistent Landscapes - unscaled)

In [38]:
# Linear - H0 - unscaled
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_unscaled_H0_train, Y_b01_unscaled_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_unscaled_H0_test, Y_b01_unscaled_H0_test))[:10])

evaluation(y_test = Y_b01_unscaled_H0_test,
           y_pred = clf_pl.predict(X_b01_unscaled_H0_test),
           filename_csv = folderpath_results + "svm_linear_H0_unscaled.csv")

Accuracy SVM : 0.56481481
Confusion-Matrix:
 [[12 25]
 [22 49]] 
 
True Negative:  12 
True Positive 49 
False Negative 22 
False Positive 25

TPR: 0.6901408450704225 
TNR:  0.32432432432432434

Accuracy: 0.5648148148148148
F1 (micro): 0.5648148148148148
F1 (macro): 0.5069451189898009


In [39]:
# Linear - H1 - unscaled
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_unscaled_H1_train, Y_b01_unscaled_H1_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_unscaled_H1_test, Y_b01_unscaled_H1_test))[:10])

evaluation(y_test = Y_b01_unscaled_H1_test,
           y_pred = clf_pl.predict(X_b01_unscaled_H1_test),
           filename_csv = folderpath_results + "svm_linear_H1_unscaled.csv")

Accuracy SVM : 0.59259259
Confusion-Matrix:
 [[16 15]
 [29 48]] 
 
True Negative:  16 
True Positive 48 
False Negative 29 
False Positive 15

TPR: 0.6233766233766234 
TNR:  0.5161290322580645

Accuracy: 0.5925925925925926
F1 (micro): 0.5925925925925926
F1 (macro): 0.5533834586466165


In [40]:
# RBF - H0 - unscaled
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_unscaled_H0_train, Y_b01_unscaled_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_unscaled_H0_test, Y_b01_unscaled_H0_test))[:10])

evaluation(y_test = Y_b01_unscaled_H0_test,
           y_pred = clf_pl.predict(X_b01_unscaled_H0_test),
           filename_csv = folderpath_results + "svm_rbf_H0_unscaled.csv")

Accuracy SVM : 0.65740740
Confusion-Matrix:
 [[ 0 37]
 [ 0 71]] 
 
True Negative:  0 
True Positive 71 
False Negative 0 
False Positive 37

TPR: 1.0 
TNR:  0.0

Accuracy: 0.6574074074074074
F1 (micro): 0.6574074074074074
F1 (macro): 0.39664804469273746


In [41]:
# RBF - H1 - unscaled
clf_pl_rbf = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_unscaled_H1_train, Y_b01_unscaled_H1_train)

print("Accuracy SVM : " + str(clf_pl_rbf.score(X_b01_unscaled_H1_test, Y_b01_unscaled_H1_test))[:10])

evaluation(y_test = Y_b01_unscaled_H1_test,
           y_pred = clf_pl_rbf.predict(X_b01_unscaled_H1_test),
           filename_csv = folderpath_results + "svm_rbf_H1_unscaled.csv")

Accuracy SVM : 0.71296296
Confusion-Matrix:
 [[ 0 31]
 [ 0 77]] 
 
True Negative:  0 
True Positive 77 
False Negative 0 
False Positive 31

TPR: 1.0 
TNR:  0.0

Accuracy: 0.7129629629629629
F1 (micro): 0.7129629629629629
F1 (macro): 0.41621621621621624
